# SC24 Reproducibility demo: 'Exploring Scalability in C++ Parallel STL Implementations'
This jupyter notebook will reserve and setup the node to reproduce the experiments in the paper "Exploring Scalability in C++ Parallel STL Implementations"
After the execution of this notebook, the user should be able to ssh into the system and execute there the scripts to reproduce the results of the paper.

In [ ]:
# Helper functions
import time

def my_log(str):
    date = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f'[{date}] {str}')

def download_fig(conn, fig):
    remotepath = f'figures/{fig}'
    localpath = f'pstl-figs/{fig}'
    my_log(f'Downloading {remotepath} into {localpath}')
    conn.get(remotepath, local=localpath)
    my_log('Done')

## Instance setup

In [ ]:
import chi
PROJECT_NAME="" # Insert your project name here
if PROJECT_NAME == "" or PROJECT_NAME == None:
    raise NameError(
        'ERROR: Variable \'PROJECT_NAME\' is empty or NULL. Give a valid value to continue!')
chi.use_site('CHI@TACC')
chi.set("project_name",PROJECT_NAME)

In [ ]:
# Prepare the variables (user, image, lease name, etc.)

from chi import server
from os import environ

# get username, but we can't use "_" in some of the next steps, so replace with "-"
USERNAME = environ.get('USER').replace("_", "-")

IMAGE_NAME = "ubuntu22-pstl" # Custom image with all software/dependencies pre-installed
IMAGE_ID = "02e6eff2-e748-4cdc-baa2-0aead7f3ab7b"
MASTER_NAME = f"sc24-pstl"
NETWORK_NAME = "sharednet1"

FLAVOR_NAME = "baremetal"
NODE_TYPE = "compute_zen3"

my_log(f'User: {USERNAME}')
my_log(f'Image: {IMAGE_NAME}')
my_log(f'Master: {MASTER_NAME}')
my_log(f'Network: {NETWORK_NAME}')
my_log(f'Flavor: {FLAVOR_NAME}')
my_log(f'Node type: {NODE_TYPE}')

SERVER_COUNT = 1

In [ ]:
# Create a reservation for your nodes
from chi import lease

reservations = []
lease.add_node_reservation(
    reservations, count=SERVER_COUNT, node_type=NODE_TYPE)

start_date, end_date = lease.lease_duration(days=1)

lease_info = lease.create_lease(f"{USERNAME}-lease",
                                reservations=reservations,
                                start_date="now",
                                end_date=end_date
                                )
lease_id = lease_info.get("id")
my_log('Waiting for the lease to be active...')
active_lease_info = lease.wait_for_active(lease_id)
my_log('Lease is active')

In [ ]:
reservation_id = lease.get_node_reservation(lease_id)
my_log(f'Reservation ID: {reservation_id}')

In [ ]:
my_log('Creating server...')
computing_node = server.create_server(
    server_name=MASTER_NAME,
    flavor_name=FLAVOR_NAME,
    image_name=IMAGE_NAME,
    image_id=IMAGE_ID,
    network_name=NETWORK_NAME,
    reservation_id=reservation_id,
)
my_log('Server created')
my_log('Waiting for node to be active...')
server.wait_for_active(computing_node.id)
my_log('Node is active')

In [ ]:
# Get a floating IP to connect to the server
computing_fip = chi.server.associate_floating_ip(computing_node.id)
my_log(f'Floating IP: {computing_fip}')
# wait for the server + sshd to become active. Especially with baremetal, it can take some time to boot
my_log('Waiting for TCP to be active')
server.wait_for_tcp(computing_fip, 22)
my_log('TCP is active')

In [ ]:
# It is possible to connect manually via SSH or...
my_log(f'Connect manually to the server with: ssh cc@{computing_fip}')

In [ ]:
# ...or connect through SSH to the server using this notebook
my_log(f"Waiting for SSH connectivity on {computing_fip} ...")
timeout = 2*60

import socket
import time

# Repeatedly try to connect via SSH.
start_time = time.perf_counter()
while True:
   try:
      with socket.create_connection((computing_fip, 22), timeout=timeout):
            my_log("Connection successful")
            break
   except OSError as ex:
      time.sleep(10)
      if time.perf_counter() - start_time >= timeout:
            my_log(f"After {timeout} seconds, could not connect via SSH. Please try again.")

In [ ]:
# Once connected, check that it works
from chi import ssh
with ssh.Remote(computing_fip) as conn:
    conn.run('ls')

## Run experiments
It is possible to run the experiments either running the scripts from a terminal (through SSH) or from Jupyter (the following cells)

In [ ]:
# We need the wand package and make sure that we can manipulate PDFs
!sudo apt update; sudo apt install imagemagick
!pip install wand
!APPENDLINE='<policy domain="coder" rights="read | write" pattern="PDF" />'
!IMFILE='/etc/ImageMagick-6/policy.xml'
!grep -qxF "$LINE" $IMFILE || sudo sed -i "/<\/policymap>/i $LINE" $IMFILE

### Run everything in a single go

In [ ]:
# Run all experiments with "00_run_all.sh", or...
from chi import ssh
from wand.image import Image as WI

figures = [
    'fig2a_for_each_its1.pdf', 'fig2b_for_each_its1000.pdf',
    'fig3a_for_each_its1.pdf', 'fig3b_for_each_its1000.pdf',
    'fig5a_incl_scan.pdf', 'fig5b_incl_scan.pdf',
    'fig7a_sort.pdf', 'fig7b_sort.pdf'
]

with ssh.Remote(computing_fip) as conn:
    conn.run('./00_run_all.sh')
    for fig in figures:
        download_fig(conn, fig)

for fig in figures:
    my_log(fig)
    display(WI(filename=f"pstl-figs/{fig}"))

### Run step by step

In [ ]:
# ...or run step by step
# 1. Compile the software
from chi import ssh
with ssh.Remote(computing_fip) as conn:
    conn.run('./01_compile.sh')

In [ ]:
# 2. Replicate Figures 2a and 2b
scripts = ['./02a_Fig2_experiments.sh', './02b_Fig2_plot.sh']
figures = ['fig2a_for_each_its1.pdf', 'fig2b_for_each_its1000.pdf']

from chi import ssh
from wand.image import Image as WI
with ssh.Remote(computing_fip) as conn:
    for script in scripts:
        conn.run(script)
    for fig in figures:
        download_fig(conn, fig)

for fig in figures:
    my_log(fig)
    display(WI(filename=f"pstl-figs/{fig}"))

In [ ]:
# 3. Replicate Figures 3a and 3b
scripts = ['./03a_Fig3_experiments.sh', './03b_Fig3_plot.sh']
figures = ['fig3a_for_each_its1.pdf', 'fig3b_for_each_its1000.pdf']

from chi import ssh
from wand.image import Image as WI
with ssh.Remote(computing_fip) as conn:
    for script in scripts:
        conn.run(script)
    for fig in figures:
        download_fig(conn, fig)

for fig in figures:
    my_log(fig)
    display(WI(filename=f"pstl-figs/{fig}"))

In [ ]:
# 4. Replicate Figures 5a and 5b
scripts = ['./04a_Fig5a_experiments.sh', './04b_Fig5a_plot.sh', './04c_Fig5b_experiments.sh', './04d_Fig5b_plot.sh']
figures = ['fig5a_incl_scan.pdf', 'fig5b_incl_scan.pdf']

from chi import ssh
from wand.image import Image as WI
with ssh.Remote(computing_fip) as conn:
    for script in scripts:
        conn.run(script)
    for fig in figures:
        download_fig(conn, fig)

for fig in figures:
    my_log(fig)
    display(WI(filename=f"pstl-figs/{fig}"))

In [ ]:
# 5. Replicate Figures 7a and 7b
scripts = ['./05a_Fig7a_experiments.sh', './05b_Fig7a_plot.sh', './05c_Fig7b_experiments.sh', './05d_Fig7b_plot.sh']
figures = ['fig7a_sort.pdf', 'fig7b_sort.pdf']

from chi import ssh
from wand.image import Image as WI
with ssh.Remote(computing_fip) as conn:
    for script in scripts:
        conn.run(script)
    for fig in figures:
        download_fig(conn, fig)

for fig in figures:
    my_log(fig)
    display(WI(filename=f"pstl-figs/{fig}"))